In [ ]:
pip install fuzzywuzzy


In [3]:
import json
import os

with open(r"C:\Users\harsh\Desktop\Rag Application\GAG_QnA1.json","r") as rr:
    data = json.loads(rr.read())
    
# Define the function to add an 'index' key to each dictionary
def add_index(d, index):
    d['index'] = index
    return d

# Create a list of dictionaries with the index added using map
indexed_dicts = list(map(lambda x: add_index(x[1], x[0]), enumerate(data.get("GAG_QnA"))))

In [5]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from concurrent.futures import ThreadPoolExecutor, as_completed

api_key = "5kkxsZLntWPg3Os7Xen2qU8S2wxQYm6Z"
model = "mistral-large-latest"

client = MistralClient(api_key=api_key)

# Define the function to add an 'index' key to each dictionary
def add_qs(d):
    short_list = d.get("question")
    chat_response = client.chat(
        model=model,
        response_format={"type": "json_object"},
        messages=[
            ChatMessage(role="user", content=f"""Just give key_components = [ ] break down the question into key components give vector representation "{short_list}" only Give python list code. Nothing else. do not give variable name and "=" sign just value. make question shorter""")
        ],
    )
    try:
        d['short_list'] = eval(chat_response.choices[0].message.content)
        return d
    except:
        return d


# # Process the dictionaries in parallel
# with ThreadPoolExecutor() as executor:
#     futures = {executor.submit(add_qs, d): d for d in indexed_dicts}
#     indexed_dicts = [future.result() for future in as_completed(futures)]




MistralAPIStatusException: Status: 429. Message: {"message":"Requests rate limit exceeded"}

In [18]:
qp = "Whats the system of charging the tax on income under Income- tax Act"

chat_response = client.chat(
        model=model,
        response_format={"type": "json_object"},
        messages=[
            ChatMessage(role="user", content=f"""Just give key_components = [ ] break down the question into key components give vector representation "{qp}" only Give python list code. Nothing else. do not give variable name and "=" sign just value. make question shorter""")
        ],
    )
keycomp = eval(chat_response.choices[0].message.content)

In [19]:
keycomp

['system', 'charging', 'tax', 'income', 'Income-tax Act']

In [7]:
import json

# Specify the file path where you want to save the JSON file
file_path = "data.json"

# Write the dictionary to a JSON file
with open(file_path, "w") as json_file:
    json.dump(indexed_dicts, json_file, indent=4)

print(f"Dictionary has been saved as JSON in {file_path}")


Dictionary has been saved as JSON in data.json


In [28]:
from fuzzywuzzy import process, fuzz
import numpy as np

# Sample list of texts
list_of_texts = ["apple", "banana", "cherry"]

# Sample dictionary with lists of texts
dict_of_texts = {
    "fruits": ["aple", "bananna", "cheery"],
    "vegetables": ["carrot", "broccoli", "spinach"],
    "snacks": ["chips", "pretzels", "cookies"],
    "beverages": ["juice", "soda", "water"]}

def fuzzy_match_lists(list1, dict2):
    results = {}
    for key, texts in dict2.items():
        scores = []
        for text in list1:
            matches = process.extract(text, texts, scorer=fuzz.partial_ratio)
            best_match = matches[0]  # Most relevant match
            scores.append(best_match[1])  # Append the score of the best match
        average_score = np.mean(scores) if scores else 0
        results[key] = {
            'average_score': average_score,
            'element_count': len(texts),
            'matches': [(text, process.extract(text, texts, scorer=fuzz.partial_ratio)[0]) for text in list1]}
    return results

def filter_and_print_results(results, threshold=5):
    # Sort results by average score in descending order
    sorted_results = sorted(results.items(), key=lambda item: item[1]['average_score'], reverse=True)
    print(sorted_results[0])
    
#     # Print results with element counts not differing too much
#     for key, value in sorted_results:
#         if abs(len(value['matches']) - value['element_count']) <= threshold:
#             print(f"Category: {key}")
#             print(f"Average Score: {value['average_score']:.2f}")
#             for text, (match, score) in value['matches']:
#                 print(f"Original: {text} -> Best Match: {match} (Score: {score})")
#             print()

# Get the fuzzy matching results
match_results = fuzzy_match_lists(list_of_texts, dict_of_texts)

# Print filtered results
filter_and_print_results(match_results)


('fruits', {'average_score': 80.33333333333333, 'element_count': 3, 'matches': [('apple', ('aple', 75)), ('banana', ('bananna', 83)), ('cherry', ('cheery', 83))]})


In [17]:
import requests

# Replace with your actual API key and endpoint
API_KEY = '5kkxsZLntWPg3Os7Xen2qU8S2wxQYm6Z'
API_ENDPOINT = 'https://api.mistral.ai/v1/prompt'

textp = "What is the meaning of \u2018India\u2019 under the Income-tax Act?"
prompt=f"""Just Give Variable key_components = [ ] break down the question into key components give vector representation "{textp}" only Give python list code. Nothing else. do not give variable name and "=" sign just value"""

# Set up the headers and payload
headers = {
    'Authorization': f'Bearer {API_KEY}',
    'Content-Type': 'application/json'
}

payload = {
    'prompt': prompt
}

# Send the request
response = requests.post(API_ENDPOINT, headers=headers, json=payload)

# Check the response
if response.status_code == 200:
    result = response.json()
    print(result)
else:
    print(f"Error: {response.status_code}")
    print(response.text)


Error: 404
{
  "message":"no Route matched with those values",
  "request_id":"6a46827ef29967119839586803527516"
}


In [57]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

api_key = "5kkxsZLntWPg3Os7Xen2qU8S2wxQYm6Z"
model = "mistral-large-latest"

client = MistralClient(api_key=api_key)

textp = "What are the main contents of the Finance Bill?"
prompt=f"""Just give key_components = [ ] break down the question into key components give vector representation "{textp}" only Give python list code. Nothing else. do not give variable name and "=" sign just value. make question shorter"""

messages = [
    ChatMessage(role="user", content=prompt)
]

chat_response = client.chat(
    model=model,
    response_format={"type": "json_object"},
    messages=messages,
)

print(type(eval(chat_response.choices[0].message.content)))


<class 'list'>


In [48]:
# pip install mistralai

In [11]:
# import os
# from mistralai.client import MistralClient
# from mistralai.models.chat_completion import ChatMessage

# api_key = "5kkxsZLntWPg3Os7Xen2qU8S2wxQYm6Z"
# model = "mistral-large-latest"

# client = MistralClient(api_key=api_key)

# # Define the function to add an 'index' key to each dictionary
# def add_qs(d, short_list):
#     chat_response = client.chat(
#         model=model,
#         response_format={"type": "json_object"},
#         messages= [
#         ChatMessage(role="user", content=f"""Just give key_components = [ ] break down the question into key components give vector representation "{short_list}" only Give python list code. Nothing else. do not give variable name and "=" sign just value. make question shorter""")
#     ],
#     )
#     try:
#         d['short_list'] = eval(chat_response.choices[0].message.content)
#         return d
#     except:
#         return d

# # Create a list of dictionaries with the index added using map
# indexed_dicts = list(map(lambda x: add_qs(x,x.get("question")), indexed_dicts))

In [12]:
indexed_dicts_filtered = list(filter(lambda x:bool(x.get("short_list")),indexed_dicts))

In [13]:
indexed_dicts_filtered

[{'page_number': '1',
  'pdf_name': 'GAG book notes-part I.pdf',
  'content': '1.1What is the date of commencement, extent and scope of the Income-tax Act,1961?\nABs.The provisions of income tax are contained in the Income-tax. Act, 196] whichextends to the whole of India and became effective from 1-4-1962 [Section 1].\nThe Act contains sections 1 to 298 and Fourteen Schedules.\nThe Income-tax Act contains provisionsfor determination of taxable income,determination of tax liability, procedure for assessment, appeals, penalties andprosecutions.It also lays down the powers and duties of various income taxauthorities.TheprovisionsregardingFacelessAssessment,FacelessAppeal,Faceless Penalty, etc. have also been incorporated in the Income Tax Act.\nFramework of the Income-tax Act: The Income-tax Act, 1961 has been divided intomany Chapters. A separate subject heading has been dealt with in each of suchChapters. Some of these Chapters have been further sub-divided and each such sub-head of th

In [14]:
mapped_dict = list(map(lambda x: {x.get("index"):x.get("short_list")},indexed_dicts_filtered))

In [15]:
mapped_dict

[{1: ['date commencement', 'extent', 'scope', 'Income-tax Act, 1961']},
 {2: ['What',
   'is',
   'meaning',
   'of',
   "'India'",
   'under',
   'Income-tax',
   'Act?']},
 {3: ['Territorial Waters',
   'Continental Shelf',
   'Exclusive Economic Zone',
   'other Maritime Zone',
   'Airspace above territory and Territorial Waters']},
 {5: ['main', 'contents', 'Finance', 'Bill']},
 {7: ['When', 'does', 'Finance', 'Bill', 'become', 'Finance', 'Act']},
 {8: ['consequences',
   '1st April',
   'assessment year',
   'Finance Act',
   'not enacted']},
 {9: ['delegated legislation', 'made', 'CBDT']},
 {15: ['person', 'Income-tax Act', 'mean']},
 {20: ['What', 'is', 'meaning', 'of', 'local', 'authority']},
 {21: ['Who', 'are', "'artificial", 'juridical', "persons'?"]},
 {23: ['key_components',
   'break down',
   'question',
   'vector representation',
   'NA']},
 {32: ['special provisions',
   'chargeability',
   'income',
   'person',
   'alienate assets',
   'avoid tax']},
 {33: ['special

In [41]:
from fuzzywuzzy import process, fuzz
import numpy as np
from pprint import pprint
# Sample list of texts
# list_of_texts = ["apple", "banana", "cherry"]

# Sample dictionary with lists of texts
# dict_of_texts = {
#     "fruits": ["aple", "bananna", "cheery"],
#     "vegetables": ["carrot", "broccoli", "spinach"],
#     "snacks": ["chips", "pretzels", "cookies"],
#     "beverages": ["juice", "soda", "water"]
# }

list_of_texts = ['system', 'charging', 'tax', 'income', 'Income-tax Act']
dict_of_texts = new_mapped_dict

def fuzzy_match_lists(list1, dict2):
    results = {}
    for key, texts in dict2.items():
        scores = []
        for text in list1:
            matches = process.extract(text, texts, scorer=fuzz.partial_ratio)
            best_match = matches[0]  # Most relevant match
            scores.append(best_match[1])  # Append the score of the best match
        average_score = np.mean(scores) if scores else 0
        results[key] = {
            'average_score': average_score,
            'element_count': len(texts),
            'matches': [(text, process.extract(text, texts, scorer=fuzz.partial_ratio)[0]) for text in list1]
        }
    return results

def filter_and_print_results(results, threshold=5):
    # Sort results by average score in descending order
    sorted_results = sorted(results.items(), key=lambda item: item[1]['average_score'], reverse=True)
    print(sorted_results[0])
    
#     # Print results with element counts not differing too much
#     for key, value in sorted_results:
#         if abs(len(value['matches']) - value['element_count']) <= threshold:
#             print(f"Category: {key}")
#             print(f"Average Score: {value['average_score']:.2f}")
#             for text, (match, score) in value['matches']:
#                 print(f"Original: {text} -> Best Match: {match} (Score: {score})")
#             print()

# Get the fuzzy matching results
# short_list = "On what rates, is the total income of the assessee taxed?The total income of the assessee is taxable at the following two rates:?"
# match_results = fuzzy_match_lists(eval(client.chat(model=model,
#         response_format={"type": "json_object"},
#         messages= [
#         ChatMessage(role="user", content=f"""Just give key_components = [ ] break down the question into key components give vector representation "{short_list}" only Give python list code. Nothing else. do not give variable name and "=" sign just value. make question shorter""")
#     ]).choices[0].message.content), normalized_dict)


match_results = fuzzy_match_lists(list_of_texts, dict_of_texts)
# pprint(match_results)
# Print filtered results
filter_and_print_results(match_results)


(82, {'average_score': 93.4, 'element_count': 17, 'matches': [('system', ('the', 67)), ('charging', ('a', 100)), ('tax', ('a', 100)), ('income', ('income?', 100)), ('Income-tax Act', ('a', 100))]})


In [44]:
import pandas as pd

In [45]:
pd.DataFrame(match_results)

,1,2,3,5,7,8,9,15,20,21,...,888,889,890,891,892,893,894,895,896,897
average_score,75.0,81.4,49.8,49.4,57.2,60.0,49.4,74.6,51.4,41.8,...,57.2,49.8,47.6,50.0,39.6,58.4,52.2,50.0,37.4,48.6
element_count,4,8,5,4,7,5,3,3,6,5,...,5,9,6,5,3,5,5,5,3,6
matches,"[(system, (date commencement, 50)), (charging,...","[(system, (is, 50)), (charging, (meaning, 57))...","[(system, (other Maritime Zone, 50)), (chargin...","[(system, (main, 40)), (charging, (main, 50)),...","[(system, (become, 33)), (charging, (Finance, ...","[(system, (assessment year, 67)), (charging, (...","[(system, (made, 40)), (charging, (delegated l...","[(system, (mean, 40)), (charging, (person, 33)...","[(system, (is, 50)), (charging, (meaning, 57))...","[(system, (persons'?, 40)), (charging, (are, 6...",...,"[(system, (Assessing Officer, 50)), (charging,...","[(system, (purchase, 50)), (charging, (purchas...","[(system, (deposited, 55)), (charging, (attrac...","[(system, (question, 33)), (charging, (NA, 50)...","[(system, (illegal business, 40)), (charging, ...","[(system, (income estimated, 50)), (charging, ...","[(system, (section 40A(3), 33)), (charging, (s...","[(system, (question, 33)), (charging, (NA, 50)...","[(system, (provisions, 36)), (charging, (provi...","[(system, (transfer, 40)), (charging, (deducti..."


In [34]:
new_mapped_dict = dict()
for each in dict_of_texts:
    val = each.items()
    new_mapped_dict.update(val.mapping)

In [36]:
# new_mapped_dict

In [16]:
indexed_dicts[14]

{'page_number': '9',
 'pdf_name': 'GAG book notes-part I.pdf',
 'content': '1.14What is the system of charging the tax on income under the Income- tax Act?\nAns.Every person, whose total income of the previous year exceeds the maximum amountwhich is not chargeable to income tax, is an assessee and chargeable to income-tax\nin Jndia at the vate or rates prescribed in the Finance Act and the Income-tax Act for\nthe relevant assessment year. However, his total income shall be determined on thebasis of his residential status in India.\nAlthough the income of previous year is subject to iIncome-tax in the assessment yearbut income-tax shall be:\n{a)deducted at source, or\n{b)paid in advance\nwhere it is so deductible or payable under any provisions of this Act.',
 'content_type': 'QnA',
 'section': 'NA',
 'section_intro': 'NA',
 'sub_section': 'NA',
 'question': 'What is the system of charging the tax on income under the Income- tax Act?',
 'index': 14}

In [77]:
from collections import defaultdict

def normalize_list_of_dicts(list_of_dicts):
    # Create a defaultdict with lists as default values
    normalized_dict = defaultdict(list)
    
    # Iterate over each dictionary in the list
    for d in list_of_dicts:
        # Iterate over each key-value pair in the dictionary
        for key, value in d.items():
            normalized_dict[key].extend(value)
    
    # Convert defaultdict to a regular dict
    return dict(normalized_dict)

# Example usage
list_of_dicts = [
    {'name': 'Alice', 'age': 25},
    {'name': 'Bob', 'age': 30},
    {'name': 'Charlie', 'age': 35}
]

normalized_dict = normalize_list_of_dicts(mapped_dict)
print(normalized_dict)



{0: ['key', 'components', 'break', 'down', 'question', 'key', 'components', 'give', 'vector', 'representation', 'NA'], 1: ['date commencement', 'extent', 'scope', 'Income-tax Act, 1961'], 2: ['What', 'is', 'meaning', 'of', "'India'", 'under', 'Income-tax', 'Act?'], 3: ['Territorial Waters', 'Continental Shelf', 'Exclusive Economic Zone', 'other Maritime Zone', 'Airspace above territory and Territorial Waters'], 4: ['study', 'understand', 'Income-tax law'], 5: ['main', 'contents', 'Finance', 'Bill'], 6: ['rates', 'total income taxed', 'two rates'], 7: ['When', 'does', 'the', 'Finance', 'Bill', 'become', 'the', 'Finance', 'Act?'], 8: ['consequences', 'April 1st', 'assessment year', 'Finance Act', 'not enacted'], 9: ['delegated legislation', 'made', 'CBDT'], 10: ['mandatory', 'place rules notifications', 'Parliament', 'which rules notifications', 'Government'], 11: ['What', 'is', 'the', 'difference', 'between', 'a', 'notification', 'and', 'a', 'circular?'], 12: ['binding effect', 'Circula

# Ruff Work

In [2]:
# list(map(lambda x: {x.get("page_number"):x.get("question")},indexed_dicts))
# list(map(lambda x: {x.get("index"):x.get("question")},data.get("GAG_QnA")))